In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
codeN = 3
Limit = 256

In [3]:
def getFile(p):
    listdir = os.listdir(p)

    # 遍历文件名数组，拼接路径
    f = []
    for i in range(len(listdir)):
        s = p + listdir[i]

        # 读入文件，切片
        v = pd.read_table(s, header=None).iloc[:, 21:24].values
        f.append(v)
    return f

In [4]:
def get_hash_name(rule):
    res = ""
    for r in rule:
        res += str(r) + "-"
    return res[:-1]

In [5]:
def train_rule_X(_v, rule):
    lenr = len(rule)
    mat = dict()
   
    div_l = np.array(rule) // codeN
    mod_l = np.array(rule) % codeN
    len_frame = max(div_l) # this is windows size.
    val = np.zeros(lenr,dtype=np.int16)
    for v in _v:
        # for each speech
        lenv = len(v)
        for i in range(lenv - len_frame):
            # for each frame
            for _t in range(lenr):
                # get index frame with rule's codeword.
                # get index frame sub.
                val[_t] = v[i + div_l[_t]][mod_l[_t]]
                # get codeword with index.
            tp = tuple(val)
            if tp in mat:
                mat[tp] += 1
            else:
                mat[tp] = 1
            
            # calc confidence of rule, this PAR.
            tp_par = tp[:-1]
            if tp_par in mat:
                mat[tp_par] += 1
            else:
                mat[tp_par] = 1
    return mat

In [6]:
def train_rules(v, rules):
    # train dataset v with rules.
    resMat = dict()
    cnt = 0
    for rule in rules:
        mat = train_rule_X(v, rule)
        hash_name = get_hash_name(rule)
        resMat[hash_name] = mat
        print('\r', cnt , "/" , len(rules), end="")
        cnt += 1
    print('\r', end="")
    return resMat

In [7]:
def get_rules_2():
    rules = []
    for i in range(codeN):
        for j in range(i + 1, codeN):
            rules.append([i, j])
    # 帧内1->1
    for i in range(codeN):
        for j in range(codeN):
            rules.append([i, j + codeN])
    return rules
    # 帧间1->1
def get_rules_3():
    rules = []
    rules.append([0, 1, 2])
    # 帧内1->2 (帧内2->1)
    for i in range(codeN):
        for j in range(codeN):
            for k in range(j + 1, codeN):
                rules.append([i, j + codeN, k + codeN])
    # 帧间1->2
    
    for i in range(codeN):
        for j in range(i + 1, codeN):
            for k in range(codeN):
                rules.append([i, j, k + codeN])
    # 帧间2->1
    return rules

def get_rules_2_and_3():
    return get_rules_2() + get_rules_3()


def get_index_from_rule(rule):
    if len(rule) == 2:
        # 帧内或帧间1->1
        if rule[1] < codeN: return 0 # 帧内1->1
        else : return 1 # 帧间1->1
    elif len(rule) == 3:
        if rule[2] < codeN: return 2 # 帧内1->2
        if rule[1] >= codeN: return 3 # 帧间1->2
        else: return 4 # 帧间2->1
        
# 从规则反解下标

def get_name_from_index(index):
    name_list = ['帧内1->1', '帧间1->1',
                 '帧内1->2','帧间1->2', '帧间2->1']
    return name_list[index]

In [8]:
def test_rule_X(_v, rule, MC, MS):
    lenr = len(rule)
    hash_name = get_hash_name(rule)
    f = np.zeros((len(_v), 2), dtype=np.int32)
    div_l = np.array(rule) // codeN
    mod_l = np.array(rule) % codeN
    len_frame = max(div_l) # this is windows size. T >= 1 is inter, T = 0 is intra.
    val = np.zeros(lenr, dtype=np.int16)
    for p in range(len(_v)):
        v = _v[p]
        # for each speech
        lenv = len(v)
        a = 0
        b = 0
        for i in range(lenv - len_frame):
            # for each frame
            for _t in range(lenr):
                # index_frame = i + div_l[_t]
                # index_frame_sub = mod_l[_t]
                # get index frame sub.
                val[_t] = v[i + div_l[_t]][mod_l[_t]]
            tp = tuple(val)
            
            t1 = tp in MC[hash_name]
            t2 = tp in MS[hash_name]
            if t1 and not t2: a += 1
            elif not t1 and t2: b += 1
        f[p][0] += a
        f[p][1] += b
    return f
    

In [10]:
def test_rules(_v, rules, MC, MS):
    fea = np.zeros((len(_v), 5, 2))
    cnt = 0
    for rule in rules:
        hash_index = get_index_from_rule(rule)
        f = test_rule_X(_v, rule, MC, MS)
        for i in range(len(f)):
            for j in range(2):
                fea[i][hash_index][j] += f[i][j]
        print('\r', cnt , "/" , len(rules), end="")
        cnt += 1
    print('\r', end="")
    return fea

In [11]:
def saveFeaWithCsv(fc, fs, name):
    res = []
    for i in range(len(fc)):
        fea_cover_line = np.append(fc[i].reshape(1, -1), 0)
        res.append(fea_cover_line)
    for i in range(len(fs)):
        fea_stego_line = np.append(fs[i].reshape(1, -1), 1)
        res.append(fea_stego_line)
        
    pd.DataFrame(res).to_csv(name, header=None, index=None)

In [12]:
import time

In [14]:
def split_v(v, split_len):
    res = []
    for _v in v:
        _x = _v[:split_len]
        res.append(_x)
    return res

In [40]:
Vc_train = split_v(getFile("h:\\dataset\\CNV_divide\\0.0\\Chinese\\train\\"), 333)
Vc_valid = split_v(getFile("h:\\dataset\\CNV_divide\\0.0\\Chinese\\valid\\"), 333)
Vc_test = split_v(getFile("h:\\dataset\\CNV_divide\\0.0\\Chinese\\test\\"), 333)
startTime = time.time()
matOfCover = None
matOfStego = None
matOfCover = train_rules(Vc_train, get_rules_2_and_3())
time.time() - startTime

904.6901962757111

In [ ]:
for i in range(1, 11):
    StartTime = time.time()
    rate_text = "1.0"
    if i < 10:
        rate_text = "0." + str(i)
    
    Vs_train = split_v(getFile("h:\\dataset\\CNV_divide\\" + rate_text + "\\Chinese\\train\\"), 333)
    Vs_valid = split_v(getFile("h:\\dataset\\CNV_divide\\" + rate_text + "\\Chinese\\train\\"), 333)
    Vs_test = split_v(getFile("h:\\dataset\\CNV_divide\\" + rate_text + "\\Chinese\\test\\"), 333)
    # read stego files.
    
    matOfStego = None
    matOfStego = train_rules(Vs_train, get_rules_all())
    print("train over.", int(time.time() - StartTime),"sec")
    # train samples.
    
    f_vc = test_rules(Vc_valid, get_rules_all(), matOfCover, matOfStego)
    print("get valid feature with cover over.", int(time.time() - StartTime),"sec")
    f_vs = test_rules(Vs_valid, get_rules_all(), matOfCover, matOfStego)
    print("get valid feature with stego over.", int(time.time() - StartTime),"sec")
    # get featrue with rules from MAT.
    saveFeaWithCsv(f_vc, f_vs, "../01 DATA/CNV/CH_VALID_RATE_ALL_" + str(i) + ".csv")

    f_tc = test_rules(Vc_test, get_rules_all(), matOfCover, matOfStego)
    print("get test feature with cover over.", int(time.time() - StartTime),"sec")
    f_ts = test_rules(Vs_test, get_rules_all(), matOfCover, matOfStego)
    print("get test feature with stego over.", int(time.time() - StartTime),"sec")
    saveFeaWithCsv(f_tc, f_ts, "../01 DATA/CNV/CH_TEST_RATE_ALL_" + str(i) + ".csv")

    # non auto rate(rate = 1)
    print("cast time=", int(time.time() - StartTime),"sec", "with CH in time(s)=", i)
    print(time.localtime())

### 短时长与极短时长的准确率

In [16]:
Vc_train_p = getFile("h:\\dataset\\CNV_divide\\0.0\\Chinese\\train\\")
Vc_valid_p = getFile("h:\\dataset\\CNV_divide\\0.0\\Chinese\\valid\\")
Vc_test_p = getFile("h:\\dataset\\CNV_divide\\0.0\\Chinese\\test\\")

In [ ]:
for i in range(1, 11):
    StartTime = time.time()
    length_text = "1.0"
    if i < 10:
        length_text = "0." + str(i)
    
    Vs_train_p = getFile("h:\\dataset\\CNV_divide\\" + length_text + "\\Chinese\\train\\")
    Vs_valid_p = getFile("h:\\dataset\\CNV_divide\\" + length_text + "\\Chinese\\valid\\")
    Vs_test_p = getFile("h:\\dataset\\CNV_divide\\" + length_text + "\\Chinese\\test\\")
    SPEECH_LEN = int(i * 100 / 3) # 10 / 3表示每个0.1s时长，100 / 3表示每个1s时长
    
    Vc_train = split_v(Vc_train_p, SPEECH_LEN)
    Vc_valid = split_v(Vc_valid_p, SPEECH_LEN)
    Vc_test = split_v(Vc_test_p, SPEECH_LEN)
    
    Vs_train = split_v(Vs_train_p, SPEECH_LEN)
    Vs_valid = split_v(Vs_valid_p, SPEECH_LEN)
    Vs_test = split_v(Vs_test_p, SPEECH_LEN)
    # read stego files.
    matOfCover = None
    matOfStego = None
    
    matOfCover = train_rules(Vc_train, get_rules_all())
    matOfStego = train_rules(Vs_train, get_rules_all())
    print("train over.")
    # train samples.
    
    f_vc = test_rules(Vc_valid, get_rules_all(), matOfCover, matOfStego)
    print("get valid feature with cover over.")

    f_vs = test_rules(Vs_valid, get_rules_all(), matOfCover, matOfStego)
    print("get valid feature with stego over.")
    # get featrue with rules from MAT.
    saveFeaWithCsv(f_vc, f_vs, "../01 DATA/CNV/CH_VALID_TIME_ALL_" + str(i) + ".csv")
    MidTime = time.time();
    f_tc = test_rules(Vc_test, get_rules_all(), matOfCover, matOfStego)
    print("get test feature with cover over.")
    f_ts = test_rules(Vs_test, get_rules_all(), matOfCover, matOfStego)
    print("get test feature with stego over.")
    saveFeaWithCsv(f_tc, f_ts, "../01 DATA/CNV/CH_TEST_TIME_ALL_" + str(i) + ".csv")
    # non auto rate(rate = 1)
    print("cast time=", int(time.time() - StartTime),"sec", "with CH in time(s)=", i)